In [ ]:
# usersPath = "data/users.txt" # Useless for this program
actionsPath = "data/actions.txt"
appsPath  = "data/apps.txt"

outputPath1 = "outPart1/"
outputPath2 = "outPart2_v2/"

In [ ]:
# Define the rdds associated with the input files

# Actions line format: UserID, AppID, Timestamp, Action
# timestamp format: YYYY/MM/DD-HH:MM:SS
actionsRDD = sc.textFile(actionsPath)

# apps line format: AppId, Name, Price, Category, Company
appsRDD = sc.textFile(appsPath)

In [ ]:
#########################################
# PART 1
#########################################

In [ ]:
# Select the actions performed in 2021
# since we are interested only in installation and removal actions, a condition
# on the type of action is also introduced

def filter2021InstallRemove(line):
    fields = line.split(",")
    action = fields[3]
    year = int(fields[2].split("/")[0])
    
    return year == 2021 and (action=="Install" or action=="Remove")

actions2021 = actionsRDD.filter(filter2021InstallRemove)

In [ ]:
# Create a PairRDD with
# key = appId, Month
# value = install/removal
# where the value is either +1 (install) or -1 (removal), depending on the type of action the line is
# associated with
#
# Then use reduceByKey to count the difference between the number of
# installations and removals of each app in a specific month of 2021


def mapAppIdMonthInstallRemoval(line):
    fields = line.split(",")
    appId = fields[1]
    month = fields[2].split("/")[1]
    action = fields[3]
    
    if (action=="Install"):
        value = 1
    else:
        value = -1
       
    return ((appId, month), value)


appMonthInstallRemoval = actions2021\
            .map(mapAppIdMonthInstallRemoval)\
            .reduceByKey(lambda v1, v2: v1 + v2)

In [ ]:
# Select only those apps and months in which #installations > #removals (i.e,
# those pairs with value>0)
appMonthWithMoreInstall = appMonthInstallRemoval\
                .filter(lambda pair: pair[1] > 0)

In [ ]:
# transform the ((appId, Month), n. installation-n. removals)) into the
# following pairRDD
# key = appId
# value = +1
# and use reduceByKey to count the number of months in 2021 that the app of
# interest (key) had #install > #removals
#
# Use then a filter to select only those apps for which count == 12.
appAll2021 = appMonthWithMoreInstall\
                    .map(lambda it: (it[0][0], 1))\
                    .reduceByKey(lambda it1, it2: it1 + it2)\
                    .filter(lambda it: it[1] == 12)

In [ ]:
# Starting from the content of Apps.txt, obtain a pairRDD with
# key = appId
# value = app name

def mapAppIDAppName(line):
    fields = line.split(",")
    appId = fields[0]
    appName = fields[1]
    
    return (appId, appName)


appIdName = appsRDD.map(mapAppIDAppName)

In [ ]:
# Join the two RDDs and keep only the fields of interest to get the final result
res1 = appAll2021.join(appIdName)\
                .map(lambda t: (t[0], t[1][1]))

In [ ]:
# Store the result in the first output folder
res1.saveAsTextFile(outputPath1)

In [ ]:
#########################################
# PART 2 - alternative version
# Less efficient than the other one because uses distinct + subtract
#########################################

In [ ]:
# Select only the "Install" actions
actionsInstall = actionsRDD\
            .filter(lambda line: line.split(",")[3]=="Install")\
            .cache()

In [ ]:
# Select installations after 2021/12/31
# Extract pairs (AppId, UserId)
# Remove duplicates
# The returned RDD contains the distinct pairs (AppId, UserId) for the apps installed
# by the users after 2021/12/31

def mapAppIdUserID(line):
    fields = line.split(",")
    userId = fields[0]
    appId = fields[1]
    
    return (appId, userId)


appsUsersAfter = actionsInstall\
                .filter(lambda line: line.split(",")[2].split("-")[0]>"2021/12/31")\
                .map(mapAppIdUserID)\
                .distinct()

In [ ]:
# Select installations before 2022/01/01
# Extract the associated pairs (AppId, UserId)
#
# The returned RDD contains the distinct pairs (AppId, UserId) for the apps installed
# by the users before 2022/01/01

appsUsersBefore = actionsInstall\
                    .filter(lambda line : line.split(",")[2].split("-")[0]<"2022/01/01")\
                    .map(mapAppIdUserID)

In [ ]:
# For each app, keep only the new users, i.e., the users who installed the app 
# for the first time after 2021/12/31.
# If a combination (appId, UserId) occurs in appsUsersBefore, that user UserIs 
# is not a new user for app appId
appsNewUsersAfter = appsUsersAfter.subtract(appsUsersBefore)

In [ ]:
# count for each app, the number of new distinct users after 31/12/2021
# Map to pairs:
# key: appId
# value: +1
# Then use reduceByKey to count
newInstall2022PerApp = appsNewUsersAfter\
                        .map(lambda pair: (pair[0], 1))\
                        .reduceByKey(lambda i1, i2: i1 + i2)\
                        .cache()

In [ ]:
# Extract the maximum number of new distinct users after 31/12/2021
maxInstallations = newInstall2022PerApp\
                            .values()\
                            .reduce(lambda i1, i2: max(i1, i2))

In [ ]:
# Select only the apps that achieved the maximum number of new distinct users
res2 = newInstall2022PerApp\
            .filter(lambda t: t[1]==maxInstallations)\
            .keys()

In [ ]:
# Store the result in the second output folder
res2.saveAsTextFile(outputPath2)